<a href="https://colab.research.google.com/github/worachot-n/Text_summarization_T5/blob/main/1_2_Text_Summarization_Custom_Rouge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

     |████████████████████████████████| 1.2 MB 7.1 MB/s 
     |████████████████████████████████| 3.8 MB 9.9 MB/s 
     |████████████████████████████████| 6.5 MB 43.7 MB/s 
     |████████████████████████████████| 895 kB 60.3 MB/s 
     |████████████████████████████████| 596 kB 56.8 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 224 kB 7.5 MB/s 
     |████████████████████████████████| 51 kB 7.1 MB/s 


In [3]:
! pip install datasets rouge-score nltk

     |████████████████████████████████| 325 kB 8.9 MB/s 
     |████████████████████████████████| 212 kB 63.4 MB/s 
     |████████████████████████████████| 136 kB 71.7 MB/s 
     |████████████████████████████████| 1.1 MB 61.7 MB/s 
     |████████████████████████████████| 127 kB 64.6 MB/s 
     |████████████████████████████████| 94 kB 118 kB/s 
     |████████████████████████████████| 271 kB 59.2 MB/s 
     |████████████████████████████████| 144 kB 60.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
from datasets import load_dataset, load_metric

In [5]:
metric = load_metric("rouge")

In [6]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Kaggle/BBC_News_Summary.csv")

In [7]:
df.sample(10)

,Unnamed: 0,articles,summaries,categories
759,759,Prodigy Monfils blows away Gaudio\n\nFrench pr...,"Curbishley said after the 4-0 win: ""There was ...",sport
1233,1233,Podcasts mark rise of DIY radio\n\nAn Apple iP...,The base station processes the information fro...,tech
1906,1906,Profits slide at India's Dr Reddy\n\nProfits a...,The Fed's Open Market Committee - which sets i...,business
391,391,February poll claim 'speculation'\n\nReports t...,But the home secretary said Mr Howard was wron...,politics
2088,2088,EU-US seeking deal on air dispute\n\nThe EU an...,"Ford, which has sold 4,000 of its first hybrid...",business
399,399,Howard 'truanted to play snooker'\n\nConservat...,Gibraltarians rejected plans for the Rock's so...,politics
1026,1026,Sony PSP console hits US in March\n\nUS gamers...,T-Mobile confirmed that the US Secret Service ...,tech
1252,1252,'Evil twin' fear for wireless net\n\nPeople us...,One in 10 adult Americans - equivalent to 22 m...,tech
1148,1148,Call for action on internet scam\n\nPhone comp...,This is likely to be in response to spammers w...,tech
1853,1853,Macy's owner buys rival for $11bn\n\nUS retail...,Malaysia's central bank is to relax restrictio...,business


In [8]:
df_business = df.loc[df['categories'] == 'business']

In [9]:
df_business = df_business[['articles', 'summaries']]
df_business

,articles,summaries
1715,EU 'too slow' on economic reforms\n\nMost EU c...,Senator Kerry attacked President Bush's econom...
1716,Glazer makes new Man Utd approach\n\nMalcolm G...,A slimming aid made from a southern African ca...
1717,Five million Germans out of work\n\nGermany's ...,Job creation was one of last year's main conce...
1718,Axa Sun Life cuts bonus payments\n\nLife insur...,Malcolm Glazer has made a fresh approach to bu...
1719,Amex shares up on spin-off news\n\nShares in A...,"If admitted into the EU, Turkey would contribu..."
...,...,...
2220,GE sees 'excellent' world economy\n\nUS behemo...,The WTO said that many developing countries su...
2221,Millions 'to lose textile jobs'\n\nMillions of...,Reuters news agency reported that Iraq's Oil M...
2222,US to rule on Yukos refuge call\n\nYukos has s...,Parmalat has sued 45 banks as it tries to recl...
2223,Business confidence dips in Japan\n\nBusiness ...,Bombardier said restructuring plans drawn up b...


In [10]:
df_business = df_business.replace(r'\n\n',' ', regex=True)
df_business

,articles,summaries
1715,EU 'too slow' on economic reforms Most EU coun...,Senator Kerry attacked President Bush's econom...
1716,Glazer makes new Man Utd approach Malcolm Glaz...,A slimming aid made from a southern African ca...
1717,Five million Germans out of work Germany's une...,Job creation was one of last year's main conce...
1718,Axa Sun Life cuts bonus payments Life insurer ...,Malcolm Glazer has made a fresh approach to bu...
1719,Amex shares up on spin-off news Shares in Amer...,"If admitted into the EU, Turkey would contribu..."
...,...,...
2220,GE sees 'excellent' world economy US behemoth ...,The WTO said that many developing countries su...
2221,Millions 'to lose textile jobs' Millions of th...,Reuters news agency reported that Iraq's Oil M...
2222,US to rule on Yukos refuge call Yukos has said...,Parmalat has sued 45 banks as it tries to recl...
2223,Business confidence dips in Japan Business con...,Bombardier said restructuring plans drawn up b...


In [11]:
df_business["summaries"] = "summarize: "+df_business["summaries"]

In [12]:
df_business.head()

,articles,summaries
1715,EU 'too slow' on economic reforms Most EU coun...,summarize: Senator Kerry attacked President Bu...
1716,Glazer makes new Man Utd approach Malcolm Glaz...,summarize: A slimming aid made from a southern...
1717,Five million Germans out of work Germany's une...,summarize: Job creation was one of last year's...
1718,Axa Sun Life cuts bonus payments Life insurer ...,summarize: Malcolm Glazer has made a fresh app...
1719,Amex shares up on spin-off news Shares in Amer...,"summarize: If admitted into the EU, Turkey wou..."


In [13]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)

In [14]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [15]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [16]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

In [17]:
# SummaryWriter takes log directory as argument
writer = SummaryWriter('/content/drive/MyDrive/Kaggle/tensorboard')

In [18]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """
  running_loss = 0.0
  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)
      writer.add_scalar('loss', loss, epoch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if _%10==0:
      writer.add_scalar('training loss', running_loss / 10, epoch * len(loader) + _)
      running_loss = 0.0

In [19]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [20]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":3,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text (article)
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text (summarize)
    "SEED": 42                     # set seed for reproducibility 

}

In [21]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)
      
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [22]:
T5Trainer(dataframe=df_business[:10], source_text="articles", target_text="summaries", model_params=model_params, output_dir="outputs")

[05:06:04] [Model]: Loading t5-base...                     <ipython-input-21-b1eef7f6be59>:14
                                                                                             

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

[05:06:48] [Data]: Reading data...                         <ipython-input-21-b1eef7f6be59>:25
                                                                                             

...

Completed 0

[05:06:55] [Validation Completed.]                         <ipython-input-21-b1eef7f6be59>:95
                                                                                             

[Model] Model saved @ outputs/model_files

[Validation] Generation on Validation data saved @ outputs/predictions.csv

[Logs] Logs saved @ outputs/logs.txt

In [23]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

In [24]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

In [25]:
eva = pd.read_csv('/content/outputs/predictions.csv')

In [26]:
eva

,Unnamed: 0,Generated Text,Actual Text
0,0,Axa reduces annual bonus payments for up to 50...,summarize: Malcolm Glazer has made a fresh app...
1,1,job cuts US phone company SBC Communications s...,summarize: Shares in India's largest power pro...


In [27]:
eva['Generated Text'].to_list()

['Axa reduces annual bonus payments for up to 50,000 with-profits investors. regular annual bonus rates on ex-Axa Equity & Law with-profits policies are to be cut from 2% to 1% for 2004. heavy stock market falls throughout 2001 and 2002 forced most firms to trim payouts.',
 "job cuts US phone company SBC Communications said it expects to cut around 12,800 jobs following its $16bn (£8.5bn) takeover of former parent AT&T. SBC said 5,125 positions would go as a result of network efficiencies, 3,400 from business operations and 2,600 across legal, advertising and public relations. the deal is subject to approval by AT&T's shareholders and regulators."]

In [28]:
!pip install rouge

In [29]:
from rouge import Rouge

In [30]:
rouge = Rouge()

In [31]:
rouge.get_scores(eva['Generated Text'].to_list(), eva['Actual Text'].to_list(), avg=True)

{'rouge-1': {'f': 0.04817541128659618,
  'p': 0.04059829059829059,
  'r': 0.05974264705882353},
 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
 'rouge-l': {'f': 0.04817541128659618,
  'p': 0.04059829059829059,
  'r': 0.05974264705882353}}

In [40]:
tensorboard --logdir='/content/drive/MyDrive/Kaggle/tensorboard'

SyntaxError: ignored